In [6]:
!pip install opencv-python

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow import keras

In [3]:
os.chdir('D:\\Загрузки\\hymenoptera_data\\train\\ants')
train_ants = os.listdir()

In [4]:
def resize(train_samples):
    array = []
    try:
        for _ in train_samples:
            image = cv2.imread(_) / 127.5 - 1     
            
            img_nrows = 400
            img_cols = 500

            img = cv2.resize(image, (img_nrows, img_cols))
            array.append(img)
    except:
        pass
    return array

In [5]:
resized_ants = resize(train_ants)

In [6]:
os.chdir('D:\\Загрузки\\hymenoptera_data\\train\\bees')
train_bees = os.listdir()

In [7]:
resized_bees = resize(train_bees)

In [8]:
y_ants = np.array([0 for i in range(len(resized_ants))])
y_bees = np.array([1 for i in range(len(resized_bees))])

y = np.concatenate((y_ants, y_bees), axis=0)
X = np.concatenate((resized_ants, resized_bees), axis=0)
X.shape

(237, 500, 400, 3)

In [9]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(500, 400, 3)),
    
    keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='same', activation='relu'),
    
    keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(2, activation='softmax')])
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 500, 400, 6)       456       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 250, 200, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 250, 200, 16)      2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 100, 16)      0         
_________________________________________________________________
flatten (Flatten)            (None, 200000)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 400002    
Total params: 402,874
Trainable params: 402,874
Non-trainable params: 0
__________________________________________________

In [48]:
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

Train on 189 samples, validate on 48 samples
Epoch 1/10
189/189 [==============================] - 21s 112ms/sample - loss: 3.5753 - acc: 0.5767 - val_loss: 1.9874 - val_acc: 0.2500
Epoch 2/10
189/189 [==============================] - 19s 102ms/sample - loss: 1.0571 - acc: 0.6455 - val_loss: 0.6906 - val_acc: 0.5833
Epoch 3/10
189/189 [==============================] - 19s 102ms/sample - loss: 0.5494 - acc: 0.7566 - val_loss: 0.7608 - val_acc: 0.3958
Epoch 4/10
189/189 [==============================] - 19s 100ms/sample - loss: 0.4246 - acc: 0.8148 - val_loss: 1.2231 - val_acc: 0.1875
Epoch 5/10
189/189 [==============================] - 19s 101ms/sample - loss: 0.3145 - acc: 0.8942 - val_loss: 1.1442 - val_acc: 0.2500
Epoch 6/10
189/189 [==============================] - 19s 102ms/sample - loss: 0.2128 - acc: 0.9524 - val_loss: 1.4218 - val_acc: 0.2292
Epoch 7/10
189/189 [==============================] - 19s 101ms/sample - loss: 0.1363 - acc: 0.9735 - val_loss: 1.2089 - val_acc: 0.2

In [11]:
os.chdir('D:\\Загрузки\\hymenoptera_data\\val\\ants')
val_ants = os.listdir()
val_res_ants = resize(val_ants)

In [12]:
os.chdir('D:\\Загрузки\\hymenoptera_data\\val\\bees')
val_bees = os.listdir()
val_res_bees = resize(val_bees)

In [13]:
X_val = np.concatenate((val_res_ants, val_res_bees), axis=0)

In [49]:
pred = model.predict(X_val)


In [22]:
def predict_res(pred):    
    predictions = []
    for i in pred:
        for _, el in enumerate(i):

            if el == max(i):
                predictions.append(_)

    y_a_val = np.array([0 for i in range(len(val_res_ants))])
    y_b_val = np.array([1 for i in range(len(val_res_bees))])

    y_val = np.concatenate((y_a_val, y_b_val), axis=0)
    
    return predictions, y_val

In [50]:
from sklearn.metrics import accuracy_score

accuracy_score(predict_res(pred)[0], predict_res(pred)[1])

0.5163398692810458

Картинок мало, потому и результат такой ), при кол-ве эпох = 5, точность получается 0.43

#### Добавляем BatchNormalozation

In [52]:
model_b = keras.models.Sequential([
    keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(500, 400, 3)),
    
    keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='same', activation='relu'),
      
    keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    keras.layers.Flatten(),
    
    keras.layers.BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True),
    
    keras.layers.Dense(2, activation='softmax')])
    
model_b.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [53]:
model_b.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 500, 400, 6)       456       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 250, 200, 6)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 250, 200, 16)      2416      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 125, 100, 16)      0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 200000)            0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 200000)            800000    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                

In [54]:
model_b.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

Train on 189 samples, validate on 48 samples
Epoch 1/10
189/189 [==============================] - 22s 116ms/sample - loss: 16.2798 - acc: 0.5503 - val_loss: 0.3749 - val_acc: 0.8958
Epoch 2/10
189/189 [==============================] - 21s 109ms/sample - loss: 9.4141 - acc: 0.7037 - val_loss: 0.8471 - val_acc: 0.7500
Epoch 3/10
189/189 [==============================] - 21s 110ms/sample - loss: 2.6676 - acc: 0.8466 - val_loss: 3.1576 - val_acc: 0.5833
Epoch 4/10
189/189 [==============================] - 21s 112ms/sample - loss: 3.2600 - acc: 0.8677 - val_loss: 12.9474 - val_acc: 0.0625
Epoch 5/10
189/189 [==============================] - 20s 106ms/sample - loss: 1.5339 - acc: 0.9206 - val_loss: 7.3536 - val_acc: 0.0625
Epoch 6/10
189/189 [==============================] - 21s 111ms/sample - loss: 0.3330 - acc: 0.9577 - val_loss: 14.7944 - val_acc: 0.0000e+00
Epoch 7/10
189/189 [==============================] - 22s 114ms/sample - loss: 0.9929 - acc: 0.8995 - val_loss: 18.6441 - val_

In [55]:
pred_b = model_b.predict(X_val)

In [56]:
accuracy_score(predict_res(pred_b)[0], predict_res(pred_b)[1])

0.49673202614379086

Лучшая попытка давала - точность, 0.575, но с таким кол-вом данных такие отклонения не имеют значение